In [3]:
#load data
from keras.datasets import mnist
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

In [4]:
#normalization
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [5]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)


In [36]:
# Mean squared error loss function
def gradient(w, b, x, y):
    m = x.shape[0]
    z = np.dot(w.T, x) + b
    a = sigmoid(z)
    d = np.dot(x, (a-y).T)
    dz = np.dot(a, (1-a).T)
    dw = np.dot(d, dz)/m            # (a-y) * a * (1 - a) * x / m
    db = np.dot(np.sum(a-y), dz)/m  # (a-y) * a * (1 - a) / m
    
    return dw, db

In [44]:
def trainer(w, b, x, y, lr, iteration):
    for i in range(iteration):   
        dw, db = gradient(w, b, x, y)
        w = w - lr * dw
        b = b - lr * db
        print("iteration = %f" % (i + 1))

    return w, b

In [38]:
#sigmoid function
def sigmoid(z):
    result = 1.0 / (1.0 + np.exp(-z))
    
    return result

In [39]:
def predict(w, b, x):
    m = x.shape[1]
    y_pred = np.zeros((1, m))
    z = np.dot(w.T, x) + b
    a = sigmoid(z)
  
    for i in range(a.shape[1]):
        if a[:, i] > 0.5: 
            y_pred[:, i] = 1    # if image is not the digit
        elif a[:, i] <= 0.5:
            y_pred[:, i] = 0    # if image is digit
    
    return y_pred

In [40]:
def accuracy(Y, y_predict):
    acc = 100.0 - np.mean(np.abs(y_predict-Y)*100.0)
    acc = round(acc, 3)
    print("Accuracy {}%".format(acc))

In [ ]:
import numpy as np

# training label and images
train_label_list = np.ones((10, train_images.shape[0], 1), dtype='float32') 
for i in range(train_images.shape[0]):
    digit = np.argmax(train_labels[i])
    train_label_list[digit][i] = 0.0
train_label_list = train_label_list.reshape(10, 1, train_images.shape[0])
train_images = train_images.T


In [ ]:
# test label and images
test_label_list = np.ones((10, test_images.shape[0], 1), dtype='float32') 
for i in range(test_images.shape[0]):
    digit = np.argmax(train_labels[i])
    test_label_list[digit][i] = 0.0
test_label_list = test_label_list.reshape(10, 1, test_images.shape[0])
test_images = test_images.T

In [ ]:
# initialize weight and bias
# w.shape -- (28 * 28, 1)
w = np.zeros((10, train_images.shape[0], 1))
b = np.zeros((10, 1))
batch_size = 32
epochs = 10

for d in range(10):  
    shuffled_indices = np.random.permutation(train_images.shape[0])
    x_shuffled = train_images[shuffled_indices]
    y_shuffled = train_label_list[d][0][shuffled_indices]
    print("Start training ", d)
    for i in range(0, train_images.shape[0], batch_size):
        x = x_shuffled[i:i+batch_size]
        y = y_shuffled[i:i+batch_size]
        w[d], b[d] = trainer(w[d], b[d], train_images, train_label_list[d], 0.5, epochs)
    print("Training {} finished".format(int(d)))
    y_pred = predict(w[d], b[d], test_images)
    accuracy(test_label_list[d], y_pred)